cleaning osm data


step1

In [19]:
import pandas as pd
import geopandas as gpd
import numpy as np

def _ensure_osmid_column(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """Make sure we have an 'osmid' column (OSMnx can put it in the index)."""
    if 'osmid' in gdf.columns:
        return gdf
    if isinstance(gdf.index, pd.MultiIndex):
        if 'osmid' in gdf.index.names:
            return gdf.reset_index()
        gdf = gdf.reset_index()
    else:
        gdf = gdf.reset_index()
    # Fallbacks
    if 'osmid' not in gdf.columns:
        if 'id' in gdf.columns:
            gdf = gdf.rename(columns={'id': 'osmid'})
        else:
            gdf['osmid'] = gdf.index.astype(str)
    return gdf

def _coalesce_cols(df: pd.DataFrame, cols):
    """Row-wise first-non-null across a list of optional columns."""
    out = pd.Series(pd.NA, index=df.index, dtype="object")
    for c in cols:
        if c in df.columns:
            out = out.fillna(df[c])
    return out

def normalize_clinic_geometries(gdf: gpd.GeoDataFrame, keep_footprint=True) -> gpd.GeoDataFrame:
    """Set CRS to EPSG:4326, convert non-points to centroids; optionally keep original footprint."""
    # CRS → 4326
    if gdf.crs is None:
        gdf = gdf.set_crs(4326, allow_override=True)
    else:
        gdf = gdf.to_crs(4326)

    if keep_footprint:
        gdf['geometry_footprint'] = gdf.geometry

    # Point or centroid
    geomtype = gdf.geometry.geom_type
    centroids = gdf.geometry.centroid
    new_geom = np.where(geomtype.eq('Point'), gdf.geometry, centroids)
    gdf = gdf.set_geometry(new_geom)
    gdf = gdf.set_crs(4326, allow_override=True)

    # Lat/Lon from point
    gdf['latitude']  = gdf.geometry.y
    gdf['longitude'] = gdf.geometry.x
    return gdf

def prune_and_rename_clinics(gdf: gpd.GeoDataFrame, keep_footprint=True) -> gpd.GeoDataFrame:
    gdf = _ensure_osmid_column(gdf).copy()

    # Filter to amenity=veterinary (if available)
    if 'amenity' in gdf.columns:
        gdf = gdf[gdf['amenity'].astype(str).str.lower().eq('veterinary')].copy()

    # Geometry normalization (point + lat/lon, keep footprint optionally)
    gdf = normalize_clinic_geometries(gdf, keep_footprint=keep_footprint)

    # Coalesce contact fields
    phone   = _coalesce_cols(gdf, ['contact:phone', 'phone'])
    website = _coalesce_cols(gdf, ['contact:website', 'website'])
    email   = _coalesce_cols(gdf, ['contact:email', 'email'])

    # Build full address
    def _full_address(row):
        a = " ".join([str(row.get('addr:street') or "").strip(),
                      str(row.get('addr:housenumber') or "").strip()]).strip()
        b = " ".join([str(row.get('addr:postcode') or "").strip(),
                      str(row.get('addr:city') or "").strip()]).strip()
        parts = [p for p in [a, b] if p]
        return ", ".join(parts) if parts else pd.NA

    gdf['full_address'] = gdf.apply(_full_address, axis=1)

    # Target schema mapping (rename/pass-through)
    rename_map = {
        'osmid': 'clinic_id',
        'name': 'clinic_name',
        'addr:street': 'street',
        'addr:housenumber': 'house_number',
        'addr:postcode': 'postcode',
        'addr:city': 'city',
        'veterinary:speciality': 'speciality',
        'wheelchair': 'wheelchair_acces',  # as requested (one 's')
        # keep: amenity, opening_hours, operator, brand, geometry
    }
    for old, new in rename_map.items():
        if old in gdf.columns:
            gdf = gdf.rename(columns={old: new})

    # Inject coalesced contact fields to target names
    gdf['phone_number'] = phone
    gdf['website']      = website
    gdf['email']        = email

    # Add district/neighbourhood placeholders (to be enriched later)
    gdf['district_id']      = pd.NA
    gdf['neighbourhood_id'] = pd.NA

    # Final column order
    final_cols = [
        'clinic_id', 'clinic_name',
        'street', 'house_number', 'postcode', 'city',
        'district_id', 'neighbourhood_id',
        'phone_number', 'website', 'email',
        'opening_hours', 'operator', 'brand',
        'speciality', 'wheelchair_acces',
        'full_address', 'latitude', 'longitude',
        'amenity', 'geometry'
    ]

    # Keep only requested columns that actually exist
    existing = [c for c in final_cols if c in gdf.columns]
    out = gdf[existing].copy()

    # Ensure proper dtypes for id/postcode if desired (keep as text here)
    if 'clinic_id' in out.columns:
        out['clinic_id'] = out['clinic_id'].astype(str)
    if 'postcode' in out.columns:
        out['postcode'] = out['postcode'].astype(str)

    return out


In [20]:
# 1) geting raw OSM features (point/polygon geometries)
from osmnx.features import features_from_place as get_features  # OSMnx ≥2
gdf_raw = get_features("Berlin, Germany", {"amenity": "veterinary"}).reset_index()

# 2) running the pruning/renaming + geometry normalization
clinics_gdf = prune_and_rename_clinics(gdf_raw, keep_footprint=True)

# 3) quick sanity checks
clinics_gdf.head(), clinics_gdf.crs, clinics_gdf.shape

clinics_gdf.to_file("berlin_vet_clinics_normalized.geojson", driver="GeoJSON")
clinics_gdf.drop(columns="geometry").to_csv("berlin_vet_clinics_normalized.csv", index=False, encoding="utf-8")


/var/folders/f2/z3f134vj17zg0vm9xq6gmy4h0000gn/T/ipykernel_58538/1492083296.py:44: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids = gdf.geometry.centroid


step2

In [21]:
# --- Apply cleaning helpers to the target-schema GeoDataFrame ---

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

def _geo_latlon(g):
    """Return (lat, lon) from geometry; centroid fallback; None if missing."""
    if g is None or g.is_empty:
        return (None, None)
    p = g
    if not isinstance(p, Point):
        try:
            p = g.centroid
        except Exception:
            return (None, None)
    return (float(p.y), float(p.x))

def clean_clinics_target_schema(clinics_gdf, country="DE", postcode_as_text=True):
    """
    Use your clean_vet_df() (which expects OSM-style column names),
    then map cleaned values back to the target schema on clinics_gdf.
    """
    gdf = clinics_gdf.copy()

    # 1) Build a temporary OSM-style frame the cleaner expects
    tmp = pd.DataFrame({
        "name": gdf.get("clinic_name"),
        "addr:street": gdf.get("street"),
        "addr:housenumber": gdf.get("house_number"),
        "addr:city": gdf.get("city"),
        "addr:postcode": gdf.get("postcode"),
        "phone": gdf.get("phone_number"),
        "website": gdf.get("website"),
        "email": gdf.get("email"),
        "opening_hours": gdf.get("opening_hours"),
        "operator": gdf.get("operator"),
        "brand": gdf.get("brand"),
        "wheelchair": gdf.get("wheelchair_acces", gdf.get("wheelchair_access")),
        "emergency": gdf.get("emergency"),
    })

    # 2) Clean using helper
    tmp_clean = clean_vet_df(tmp, country=country, postcode_as_text=postcode_as_text)

    # 3) Write cleaned values back to target-schema columns
    gdf["clinic_name"]   = tmp_clean["name"]
    gdf["street"]        = tmp_clean["addr:street"]
    gdf["house_number"]  = tmp_clean["addr:housenumber"]
    gdf["city"]          = tmp_clean["addr:city"]
    gdf["postcode"]      = tmp_clean["addr:postcode"]
    gdf["phone_number"]  = tmp_clean.get("phone")
    gdf["website"]       = tmp_clean.get("website")
    gdf["email"]         = tmp_clean.get("email")
    gdf["opening_hours"] = tmp_clean.get("opening_hours")
    gdf["operator"]      = tmp_clean.get("operator")
    gdf["wheelchair_acces"] = tmp_clean.get("wheelchair")  # keep your column name
    gdf["emergency"]        = tmp_clean.get("emergency")

    # Optional: bring over the validity flag from your helper
    if "opening_hours_valid" in tmp_clean.columns:
        gdf["opening_hours_valid"] = tmp_clean["opening_hours_valid"]

    # 4) Rebuild full_address (street + house + postcode + city) when available
    def _addr_row(r):
        parts = [r.get("street"), r.get("house_number"), r.get("postcode"), r.get("city")]
        parts = [p for p in parts if isinstance(p, str) and p.strip() != ""]
        return ", ".join(parts) if parts else None
    gdf["full_address"] = gdf.apply(_addr_row, axis=1)

    # 5) Ensure geometry CRS and lat/lon columns exist (point or centroid)
    if getattr(gdf, "crs", None) is None:
        try:
            gdf.set_crs(epsg=4326, inplace=True, allow_override=True)
        except Exception:
            pass
    elif gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(4326)

    if "latitude" not in gdf.columns or "longitude" not in gdf.columns:
        lats, lons = zip(*gdf.geometry.map(_geo_latlon))
        gdf["latitude"] = lats
        gdf["longitude"] = lons

    # 6) (Safety) keep only veterinary amenity rows if column present
    if "amenity" in gdf.columns:
        gdf = gdf[gdf["amenity"].fillna("").astype(str).str.lower().eq("veterinary")]

    return gdf

# --- run it ---
clinics_clean = clean_clinics_target_schema(clinics_gdf, country="DE", postcode_as_text=True)

# quick peek / save
display(clinics_clean.head())
clinics_clean.drop(columns="geometry").to_csv("berlin_vet_clinics_clean.csv", index=False, encoding="utf-8")


,clinic_id,clinic_name,street,house_number,postcode,city,district_id,neighbourhood_id,phone_number,website,...,opening_hours,operator,wheelchair_acces,full_address,latitude,longitude,amenity,geometry,emergency,opening_hours_valid
0,268917040,Tierarztpraxis am Urban,Baerwaldstraße,69,10961,Berlin,<NA>,<NA>,None,None,...,"Mo-Sa 10:00-12:00, Mo 17:00-19:00, Tu,We,Fr 16...",None,no,"Baerwaldstraße, 69, 10961, Berlin",52.495684,13.405233,veterinary,POINT (13.40523 52.49568),None,None
1,299795048,Dr. med. vet. Elke Hartwig,Straße 48,67,13125,Berlin,<NA>,<NA>,+49309437820,http://www.tierarztpraxis-hartwig.de/,...,"Mo,Tu,Th,Fr 10:00-12:00, Mo-Fr 15:00-18:00",None,limited,"Straße 48, 67, 13125, Berlin",52.606286,13.479555,veterinary,POINT (13.47955 52.60629),None,None
2,347294456,Tierarztpraxis Dr. Bernhard Sörensen,Königsberger Straße,36,12207,Berlin,<NA>,<NA>,+49307738321,https://www.tierarztpraxis-soerensen.de/,...,"Mo-Fr 09:00-20:00; Sa, Su 10:00-18:00",None,yes,"Königsberger Straße, 36, 12207, Berlin",52.429722,13.320133,veterinary,POINT (13.32013 52.42972),None,None
3,394867279,Tierarztpraxis Jeanette Koepsel,None,None,None,None,<NA>,<NA>,None,None,...,None,None,None,None,52.535199,13.270573,veterinary,POINT (13.27057 52.5352),None,None
4,411550894,Kleintierarztpraxis Berlin Kaulsdorf,Planitzstraße,19,12621,Berlin,<NA>,<NA>,+493053018585,https://www.tierarzt-kaulsdorf.de/,...,"Mo-Fr 09:00-19:00 open ""tel. Terminvereinbarun...",Dr. Berit Miels;Dr. Mathias Kochert,None,"Planitzstraße, 19, 12621, Berlin",52.509511,13.589635,veterinary,POINT (13.58964 52.50951),None,None


In [22]:
# dedupe on raw OSM features
gdf_raw = _ensure_osmid_column(gdf_raw)
before = len(gdf_raw)
gdf_raw = gdf_raw.drop_duplicates(subset=["osmid"], keep="first").copy()
print(f"Dropped {before - len(gdf_raw)} exact duplicates on osmid")


Dropped 0 exact duplicates on osmid


In [23]:
before = len(clinics_gdf)
clinics_gdf = clinics_gdf.drop_duplicates(subset=["clinic_id"], keep="first").copy()
print(f"Dropped {before - len(clinics_gdf)} exact duplicates on clinic_id")


Dropped 0 exact duplicates on clinic_id
